In [2]:
from openai import OpenAI
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.beta.assistant_stream_event import ThreadMessageDelta
from openai.types.beta.threads.text_delta_block import TextDeltaBlock 
import os
import pandas as pd

In [4]:
df_scrape = pd.read_pickle('/app/Store/dataframe.pkl')
df_asst = pd.read_csv('/app/Store/assistant_files.csv')

df_asst.rename(columns={'filename': 'file_path'}, inplace=True)

# Merging on the common column 'key'
df_merge = pd.merge(df_scrape, df_asst, on='file_path', how='inner')

In [ ]:

entry = {
    "url": ,
    "text_hash":,
    "date_accessed": , 
    "file_path": ,
    "file_id": ,
    "vectorstore_id": , 
    "assistant_id": ,

}